<a href="https://colab.research.google.com/github/NohmanAudi/Indeed_Job_Posts/blob/main/Job_postings_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Scraping code source: https://medium.com/codex/web-scraping-with-beautifulsoup-66a3a2b3b60
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import timedelta, datetime

In [2]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
link="https://de.indeed.com/jobs?q=Data+Analyst&l=Berlin&start=10"

""" Sending HTTP Request"""
r=requests.get(link,headers=headers)

"""Check response status Code"""
r.status_code

200

In [3]:
"""URL Template"""
url_temp= "https://de.indeed.com/jobs?q={}&l={}&fromage=30&start={}" #### changing url_temp to specify the postend time to 30 days as indeed doesn't show specific date beyond
base_link="https://de.indeed.com"

"""This function takes the URL template, designation and city as inputs.
It navigates through the top 200 search results and scans all the <a> tags and returns a list of 
all the href attributes."""

def get_href(url_temp,position,city):
    results_per_city=200
    href_list=[]
    for start in range(0,results_per_city+10,10):
        url=url_temp.format(position,city,start)
        r=requests.get(url,headers=headers)
        soup=BeautifulSoup(r.text,"html.parser")    
       
        for i in soup.find_all('a'):
            # if tag has attribute of class
            if i.has_attr( "href" ):
                k=i['href']
                href_list.append(base_link+k)
    
    return href_list

In [4]:
"""This function takes the list of all the href attributes as input, 
finds the URLs with the mentioned strings and returns a list of those URLs."""

def get_job_links(href_list):
    job_links=[]
    for a in href_list:
        if a.find('/rc/clk')!=-1:
            job_links.append(a)
        elif a.find('/company/')!=-1:
            job_links.append(a)
    return job_links

In [5]:
"""This function takes the list of the URLs of the job postings and the city and does the following:
1. Send HTTP request to each of the URL.
2. Creates a soup object with html parsing.
3. Extracts title, company name, location and job description from each of the webpage and returns a dataframe."""



re_reqmnt = ['experience', 'knowledge', 'skills', 'strong', 'ability', 'years', 'good']



def get_job_df(job_links,city):
    df=pd.DataFrame(columns=[ "date", "job_location", "job_title", "company", "company_alt","requirement", "add_req", "job_description"])
    
    for i in job_links:
        req=requests.get(i,headers=headers)
        soup_req=BeautifulSoup(req.text,"html.parser")
        try:
            title=soup_req.find('h1',{'class': 'icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title'}).text
        except:
            continue
        try:
            company=soup_req.find('div',{'class':'icl-u-lg-mr--sm icl-u-xs-mr--xs'}).find_next().text #this worked for one company, trying for all
        except:
            continue
        
        company_alt=soup_req.find('div',{'class':'jobsearch-JobMetadataFooter'}).find_next().text #grabbing comapny name from footer


        try:
            location=soup_req.find('div',{'class':'jobsearch-InlineCompanyRating icl-u-xs-mt--xs jobsearch-DesktopStickyContainer-companyrating'}).text
        except:
            location=city
        
        try:
            sections = pd.Series([x.text.strip() for x in soup_req.find("div", {'class':'jobsearch-jobDescriptionText'}).find_all('ul')])
            see = sections.str.lower().str.split().apply(lambda words: [" ".join(word for word in words if word in re_reqmnt)])
            see_idx = see.apply(lambda x: x[0]).replace("", np.nan).dropna().index
            qualifications = sections[see_idx].reset_index(drop=True).sort_values(ascending=False)
            if len(qualifications) > 1:
              qual = qualifications[0]
              add_qual = qualifications[1]
            elif len(qualifications) == 1:
              qual = qualifications[0]
              add_qual = np.nan
            else:
              qual = np.nan
              add_qual = np.nan

        except:
          continue

        try:
            desc=soup_req.find('div',{'class':'jobsearch-jobDescriptionText'}).text
        except:
            continue
        date=soup_req.find('div',{'class':'jobsearch-JobMetadataFooter'}).find_next().find_next().text #this works

        df = df.append({"job_location":city, "job_title":title, "company":company, "job_description":desc, "company_alt": company_alt, "date": date,
                "requirement": qual, "add_req": add_qual}, ignore_index=True)


    
    return df

In [6]:
"""Calling all the above functions inside this function which takes the URL template, designation and city as inputs."""

def get_job_postings(url_temp,position,city):
    
    href_list= get_href(url_temp,position,city)
    
    job_links= get_job_links(href_list)
    
    job_df= get_job_df(job_links,city)
    
    return job_df

In [7]:
data = get_job_postings(url_temp,position='Data+Analyst',city='Berlin')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [8]:
data.to_csv('job_data.csv')

In [9]:
data_analyst_df = pd.read_csv('/content/job_data.csv', index_col=0)
data_analyst_df.head()

,date,job_location,job_title,company,company_alt,requirement,add_req,job_description
0,.css-fjuv6g{box-sizing:border-box;background:n...,Berlin,Business Data Analyst – Level 1 (m/f/d),ICE International Copyright Enterprise Germany...,vor 22 Tagen,"You’re keen to delve into the world of coding,...",NaN,What is ICE?ICE is the result of a big vision ...
1,.css-fjuv6g{box-sizing:border-box;background:n...,Berlin,Product Data Analyst (m/f/d),COMATCH GmbH,vor 8 Tagen,You have significant experience (3+ years) in ...,Top Employer Award - awarded several times by ...,"\nYour missionAs COMATCH and Malt, we are now ..."
2,vor 8 Tagen,Berlin,"Claims & Risk Analyst, EMEA (m/w/d)",Tesla,Tesla,4 to 6 years of multi-line insurance/claims ma...,NaN,\n\n\n\n\nThe Role\n\n\n\n Tesla’s Global Risk...
3,.css-fjuv6g{box-sizing:border-box;background:n...,Berlin,Senior Data Analyst (m/f/x),Getsafe GmbH,vor 28 Tagen,You will develop further our BI and reporting ...,You have 4+ years of experience in Business In...,\nGetsafe is one of the fastest growing tech c...
4,vor 2 Tagen,Berlin,Senior Data Analyst - Global Data (f/m/d),Delivery Hero,Delivery Hero,Want to be a Hero? Join the #1 GLOBAL LEADER i...,More than 4 years experience as a BI/Data Anal...,\n\n\n\n\n\n Want to be a Hero? Join the ...


In [10]:
data_analyst_df.shape

(156, 8)

In [32]:
!pip install langdetect #Language detection library ported from Google's language-detection.
from langdetect import detect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 8.5 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=bd0065c3d8dd70d1a657b5c3db81a00d0913d4af38591f9b7ff25e4dba711683
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect


In [48]:
#detecting records with German language and dropping them
idx_ger_records = data_analyst_df[data_analyst_df.job_description.apply(lambda x: detect(x)) == 'de'].index
data_analyst_df.drop(index=idx_ger_records, inplace=True)

In [51]:
#records in requirement with missings values
data_req_na = data_analyst_df[data_analyst_df.requirement.isna()]
data_req_na

,date,job_location,job_title,company,requirement,add_req,job_description
58,2022-05-20,Berlin,Pricing Analyst / Pricing Analyst Manager (m/w/d),Oda,NaN,NaN,\n\nOda is a company on the move. We built Nor...
111,2022-05-07,Berlin,Senior Data Engineer,Oxygen Digital Recruitment,NaN,NaN,\n\nWe're working with a great Green-Energy co...
128,2022-05-17,Berlin,UI Developer - Berlin,ION,NaN,NaN,\n\n\nThe Role:\n\n\n\n Do you want to join th...
135,2022-05-17,Berlin,UI Developer - Berlin,ION,NaN,NaN,The Role:\nDo you want to join the visionary i...
155,2022-05-04,Berlin,ERP Functional Consultant,"Arrow Electronics, Inc.",NaN,NaN,\n\nPosition:\n ERP Functional Consultant\n \...


In [ ]:
data_req_na.job_description.values[0]

In [ ]:
data_req_na.job_description.values[0].split('\n\n')

In [12]:
data_analyst_df.date.str.startswith('.css').value_counts()

False    141
True      15
Name: date, dtype: int64

In [13]:
data_analyst_df.date.isna().any()

False

In [14]:
data_analyst_df[data_analyst_df.date.str.startswith('.css')].date.index

Int64Index([0, 1, 3, 8, 9, 10, 17, 23, 24, 26, 31, 42, 45, 60, 93], dtype='int64')

In [15]:
#index_rep = data_analyst_df[data_analyst_df.date == "Diesen Job melden"].date.index #bad values for old de.indeed format
data_analyst_df.date.fillna('.css', inplace=True) #found one record at one attempt to have NaN. Filling it with .css to preserve the structure
index_rep = data_analyst_df[data_analyst_df.date.str.startswith('.css')].index #bad values for new de.indeed format

In [16]:
#footers had it wrong sometimes, replaced with company_alt that had the right date which exist in the footer as well

actual_date = data_analyst_df.iloc[index_rep].company_alt
fake_date = data_analyst_df.iloc[index_rep].date
data_analyst_df.date.replace(fake_date.values, actual_date.values, inplace=True)

In [17]:
data_analyst_df.date.value_counts()

vor 8 Tagen     27
vor 4 Tagen     12
vor 25 Tagen    11
vor 18 Tagen    10
vor 7 Tagen      9
vor 22 Tagen     9
vor 17 Tagen     7
vor 15 Tagen     7
vor 21 Tagen     6
vor 3 Tagen      6
vor 16 Tagen     6
vor 29 Tagen     5
vor 23 Tagen     5
vor 2 Tagen      5
vor 14 Tagen     4
Heute            4
vor 9 Tagen      4
vor 24 Tagen     4
vor 11 Tagen     3
vor 28 Tagen     3
vor 10 Tagen     3
vor 19 Tagen     2
vor 6 Tagen      2
vor 5 Tagen      1
vor 1 Tag        1
Name: date, dtype: int64

In [18]:
data_analyst_df.company.value_counts()

Delivery Hero                  11
Zalando                         7
Deutsche Bank                   5
Aurora Energy Research GmbH     4
Lieferando                      4
                               ..
HelloFresh                      1
Yelp                            1
Point Nine                      1
Mambu                           1
HERE Technologies               1
Name: company, Length: 100, dtype: int64

In [19]:
ind = data_analyst_df[data_analyst_df.company == ""].company.index

In [20]:
#This time getting the company from the footer
actual = data_analyst_df.iloc[ind].company_alt
fake = data_analyst_df.iloc[ind].company
data_analyst_df.company.replace(fake.values, actual.values, inplace=True)

In [21]:
data_analyst_df.drop(columns='company_alt', inplace=True) #dropping company_alt after extracting actual date and company values

In [22]:
data_analyst_df.company.value_counts()

Delivery Hero                  11
Zalando                         7
Deutsche Bank                   5
Aurora Energy Research GmbH     4
Lieferando                      4
                               ..
HelloFresh                      1
Yelp                            1
Point Nine                      1
Mambu                           1
HERE Technologies               1
Name: company, Length: 100, dtype: int64

In [23]:
data_analyst_df.head()

,date,job_location,job_title,company,requirement,add_req,job_description
0,vor 8 Tagen,Berlin,Business Data Analyst – Level 1 (m/f/d),ICE International Copyright Enterprise Germany...,"You’re keen to delve into the world of coding,...",NaN,What is ICE?ICE is the result of a big vision ...
1,vor 8 Tagen,Berlin,Product Data Analyst (m/f/d),COMATCH GmbH,You have significant experience (3+ years) in ...,Top Employer Award - awarded several times by ...,"\nYour missionAs COMATCH and Malt, we are now ..."
2,vor 8 Tagen,Berlin,"Claims & Risk Analyst, EMEA (m/w/d)",Tesla,4 to 6 years of multi-line insurance/claims ma...,NaN,\n\n\n\n\nThe Role\n\n\n\n Tesla’s Global Risk...
3,vor 8 Tagen,Berlin,Senior Data Analyst (m/f/x),Getsafe GmbH,You will develop further our BI and reporting ...,You have 4+ years of experience in Business In...,\nGetsafe is one of the fastest growing tech c...
4,vor 2 Tagen,Berlin,Senior Data Analyst - Global Data (f/m/d),Delivery Hero,Want to be a Hero? Join the #1 GLOBAL LEADER i...,More than 4 years experience as a BI/Data Anal...,\n\n\n\n\n\n Want to be a Hero? Join the ...


In [24]:
#date posted to actual date
data_analyst_df.date.replace(['Heute', 'Gerade geschaltet'], 0, inplace=True) #replacing date "Heute": today and "Gerade geschaltet":Just posted, with 0
data_analyst_df.date.replace(regex=[r'\D+'], value="", inplace=True) #remove non digits
data_analyst_df.date = data_analyst_df.date.astype(int).apply(lambda x: datetime.now().date()-timedelta(x)) #calculating the actual date

In [25]:
data_analyst_df.head()

,date,job_location,job_title,company,requirement,add_req,job_description
0,2022-05-20,Berlin,Business Data Analyst – Level 1 (m/f/d),ICE International Copyright Enterprise Germany...,"You’re keen to delve into the world of coding,...",NaN,What is ICE?ICE is the result of a big vision ...
1,2022-05-20,Berlin,Product Data Analyst (m/f/d),COMATCH GmbH,You have significant experience (3+ years) in ...,Top Employer Award - awarded several times by ...,"\nYour missionAs COMATCH and Malt, we are now ..."
2,2022-05-20,Berlin,"Claims & Risk Analyst, EMEA (m/w/d)",Tesla,4 to 6 years of multi-line insurance/claims ma...,NaN,\n\n\n\n\nThe Role\n\n\n\n Tesla’s Global Risk...
3,2022-05-20,Berlin,Senior Data Analyst (m/f/x),Getsafe GmbH,You will develop further our BI and reporting ...,You have 4+ years of experience in Business In...,\nGetsafe is one of the fastest growing tech c...
4,2022-05-26,Berlin,Senior Data Analyst - Global Data (f/m/d),Delivery Hero,Want to be a Hero? Join the #1 GLOBAL LEADER i...,More than 4 years experience as a BI/Data Anal...,\n\n\n\n\n\n Want to be a Hero? Join the ...


Getting some css in the date field.
Needs a fix! SOLVED ✔

Convert date from date posted to actual date SOLVED ✔

**Refactoring**

*   Refactor get_job_df function. Move the sections logic to another function.

*   Refactor company and date fixes by using functions



Add feature, company rating from (glassdoor? indeed? kununu?)

Add experience level feature

check which companies add years of experience that unmatch the experience level in the job post #HR accountability